In [1]:
import pandas as pd 
import sqlite3
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
df_chicago_public_schools = pd.read_csv('ChicagoPublicSchools.csv')
df_crime = pd.read_csv('ChicagoCrimeData.csv')
df_census = pd.read_csv('ChicagoCensusData.csv')

In [3]:
df_crime.head()

,ID,CASE_NUMBER,DATE,BLOCK,IUCR,PRIMARY_TYPE,DESCRIPTION,LOCATION_DESCRIPTION,ARREST,DOMESTIC,...,DISTRICT,WARD,COMMUNITY_AREA_NUMBER,FBICODE,X_COORDINATE,Y_COORDINATE,YEAR,LATITUDE,LONGITUDE,LOCATION
0,3512276,HK587712,2004-08-28,047XX S KEDZIE AVE,890,THEFT,FROM BUILDING,SMALL RETAIL STORE,False,False,...,9,14.0,58.0,6,1155838.0,1873050.0,2004,41.807440,-87.703956,"(41.8074405, -87.703955849)"
1,3406613,HK456306,2004-06-26,009XX N CENTRAL PARK AVE,820,THEFT,$500 AND UNDER,OTHER,False,False,...,11,27.0,23.0,6,1152206.0,1906127.0,2004,41.898280,-87.716406,"(41.898279962, -87.716405505)"
2,8002131,HT233595,2011-04-04,043XX S WABASH AVE,820,THEFT,$500 AND UNDER,NURSING HOME/RETIREMENT HOME,False,False,...,2,3.0,38.0,6,1177436.0,1876313.0,2011,41.815933,-87.624642,"(41.815933131, -87.624642127)"
3,7903289,HT133522,2010-12-30,083XX S KINGSTON AVE,840,THEFT,FINANCIAL ID THEFT: OVER $300,RESIDENCE,False,False,...,4,7.0,46.0,6,1194622.0,1850125.0,2010,41.743665,-87.562463,"(41.743665322, -87.562462756)"
4,10402076,HZ138551,2016-02-02,033XX W 66TH ST,820,THEFT,$500 AND UNDER,ALLEY,False,False,...,8,15.0,66.0,6,1155240.0,1860661.0,2016,41.773455,-87.706480,"(41.773455295, -87.706480471)"


In [4]:
conn=sqlite3.connect("sql_database_assignment_ibm.db") #create database

In [5]:
try:
    df_crime.to_sql('crime', conn)
except ValueError:
    print("La tabla ya existe.")

La tabla ya existe.


In [6]:
try:
    df_chicago_public_schools.to_sql('schools', conn)
except ValueError:
    print("La tabla ya existe.")

La tabla ya existe.


C:\Users\erica\anaconda3\lib\site-packages\pandas\core\generic.py:2779: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  sql.to_sql(


In [7]:
try:
    df_census.to_sql('census',conn)
except ValueError:
    print("La tabla ya existe.")

La tabla ya existe.


In [8]:
%load_ext sql
%sql sqlite:///sql_database_assignment_ibm.db

# Problema1

### Problem 1: Find the total number of crimes recorded in the CRIME table.

In [9]:
%%sql
SELECT COUNT(*)  FROM crime

 * sqlite:///sql_database_assignment_ibm.db
Done.


COUNT(*)
533


In [10]:
df_chicago_public_schools = df_chicago_public_schools.rename(columns={"Elementary, Middle, or High School":"elementary_or_high_school"})

## P2
### Problem 2: List community areas with per capita income less than 11000.

In [11]:
%%sql

SELECT COMMUNITY_AREA_NAME, PER_CAPITA_INCOME AS PER_CAPITA_INCOME 
FROM census WHERE PER_CAPITA_INCOME < 11000

 * sqlite:///sql_database_assignment_ibm.db
Done.


COMMUNITY_AREA_NAME,PER_CAPITA_INCOME
West Garfield Park,10934
South Lawndale,10402
Fuller Park,10432
Riverdale,8201


## P3
### Problem 3: List all case numbers for crimes involving minors?

In [12]:
%%sql

SELECT * from crime WHERE DESCRIPTION LIKE '%minor%'

 * sqlite:///sql_database_assignment_ibm.db
Done.


index,ID,CASE_NUMBER,DATE,BLOCK,IUCR,PRIMARY_TYPE,DESCRIPTION,LOCATION_DESCRIPTION,ARREST,DOMESTIC,BEAT,DISTRICT,WARD,COMMUNITY_AREA_NUMBER,FBICODE,X_COORDINATE,Y_COORDINATE,YEAR,LATITUDE,LONGITUDE,LOCATION
514,3987219,HL266884,2005-03-31,024XX N CLARK ST,2210,LIQUOR LAW VIOLATION,SELL/GIVE/DEL LIQUOR TO MINOR,CONVENIENCE STORE,1,0,2333,19,43.0,7.0,22,1172680.0,1916483.0,2005,41.92626872,-87.64089934,"(41.926268719, -87.640899336)"
515,3266814,HK238408,2004-03-13,093XX S STONY ISLAND AVE,2230,LIQUOR LAW VIOLATION,ILLEGAL CONSUMPTION BY MINOR,ALLEY,1,0,413,4,8.0,48.0,22,1188539.0,1843379.0,2004,41.72530099,-87.58496589,"(41.72530099, -87.584965887)"


## P4

### Problem 4: List all kidnapping crimes involving a child?(children are not considered minors for the purposes of crime analysis)

In [13]:
%%sql

SELECT * FROM crime WHERE PRIMARY_TYPE = 'KIDNAPPING' AND DESCRIPTION LIKE '%child%'


 * sqlite:///sql_database_assignment_ibm.db
Done.


index,ID,CASE_NUMBER,DATE,BLOCK,IUCR,PRIMARY_TYPE,DESCRIPTION,LOCATION_DESCRIPTION,ARREST,DOMESTIC,BEAT,DISTRICT,WARD,COMMUNITY_AREA_NUMBER,FBICODE,X_COORDINATE,Y_COORDINATE,YEAR,LATITUDE,LONGITUDE,LOCATION
520,5276766,HN144152,2007-01-26,050XX W VAN BUREN ST,1792,KIDNAPPING,CHILD ABDUCTION/STRANGER,STREET,0,0,1533,15,29.0,25.0,20,1143050.0,1897546.0,2007,41.87490841,-87.75024931,"(41.874908413, -87.750249307)"


## P5
### Problem 5: What kind of crimes were recorded at schools?

In [14]:
%%sql

SELECT DISTINCT(PRIMARY_TYPE) FROM crime WHERE LOCATION_DESCRIPTION LIKE '%SCHOOL%'


 * sqlite:///sql_database_assignment_ibm.db
Done.


PRIMARY_TYPE
BATTERY
CRIMINAL DAMAGE
NARCOTICS
ASSAULT
CRIMINAL TRESPASS
PUBLIC PEACE VIOLATION


## P6
### Problem 6: List the average safety score for all types of schools.

In [15]:
%%sql
select elementary_or_high_school, avg(SAFETY_SCORE) as average_safety_score from SCHOOLS

 * sqlite:///sql_database_assignment_ibm.db
Done.


elementary_or_high_school,average_safety_score
ES,49.50487329434698


## P7
### Problem 7: List 5 community areas with highest % of households below poverty line

In [16]:
%%sql

SELECT COMMUNITY_AREA_NAME, PERCENT_HOUSEHOLDS_BELOW_POVERTY FROM CENSUS
ORDER BY PERCENT_HOUSEHOLDS_BELOW_POVERTY DESC LIMIT 5

 * sqlite:///sql_database_assignment_ibm.db
Done.


COMMUNITY_AREA_NAME,PERCENT_HOUSEHOLDS_BELOW_POVERTY
Riverdale,56.5
Fuller Park,51.2
Englewood,46.6
North Lawndale,43.1
East Garfield Park,42.4


## P8
### Problem 8: Which community area(number) is most crime prone?

In [17]:
%%sql 
SELECT COMMUNITY_AREA_NUMBER from CRIME 
group by COMMUNITY_AREA_NUMBER order by COUNT(*) DESC NULLS LAST 
LIMIT 1

 * sqlite:///sql_database_assignment_ibm.db
Done.


COMMUNITY_AREA_NUMBER
25.0


## P9
### Problem 9: Use a sub-query to find the name of the community area with highest hardship index.

In [18]:
%%sql 
SELECT community_area_name FROM census
WHERE hardship_index = (SELECT max(hardship_index) FROM census)

 * sqlite:///sql_database_assignment_ibm.db
Done.


COMMUNITY_AREA_NAME
Riverdale


## P10
### Problem 10: Use a sub-query to determine the Community Area Name with most number of crimes?

In [19]:
%%sql 
SELECT COMMUNITY_AREA_NAME FROM CENSUS 
WHERE COMMUNITY_AREA_NUMBER IN (SELECT COMMUNITY_AREA_NUMBER from CRIME
                                group by COMMUNITY_AREA_NUMBER 
                                order by COUNT(*) DESC LIMIT 1)


 * sqlite:///sql_database_assignment_ibm.db
Done.


COMMUNITY_AREA_NAME
Austin
